#### **Chains**  
Chains are a way to connect multiple steps (components) into a pipeline.Each step’s output becomes the next step’s input, The entire flow becomes automated.  
You only need to provide input to the first step, and LangChain handles the rest.

**Why Use Chains?** 
1) Reduces repetitive code
2) Improves modularity
3) Automates end-to-end execution
4) Easier to maintain and scale

**Types of Chains**  
**1) Sequential Chains** - Linear Steps, one after another(like a pipeline)  
**2) Parallel Chains** - Multiple chains run at the same time (for parallel processing)  
**3) Conditional Chains** - Based on Input, chains are triggered (like if/else logic flow)

In [1]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

load_dotenv()

c:\Users\Aryan\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

**Simple Chain**

In [2]:
prompt = PromptTemplate(
    template="Generate 5 interesting facts about {topic}",
    input_variables=["topic"]
)

model = ChatGroq(model="llama-3.3-70b-versatile")

parser = StrOutputParser()

chain = prompt | model | parser
result = chain.invoke({'topic': 'cricket'})
print(result)

Here are five interesting facts about cricket:

1. **The longest game in history**: The longest cricket match in history was played between England and South Africa in 1939, lasting for 14 days. The game was eventually abandoned due to weather conditions and the fact that the English team had to catch a boat to return home.

2. **The origins of the term "over"**: In cricket, an "over" refers to a set of six deliveries bowled by a bowler. The term originated from the fact that the umpire would call out "over" when the bowler had completed six deliveries, indicating that the bowler had to switch ends and the other team's bowler would take over.

3. **The first cricket World Cup**: The first Cricket World Cup was held in 1975 in England, with eight teams participating. The West Indies won the tournament, defeating Australia in the final at Lord's Cricket Ground in London.

4. **The fastest delivery in cricket history**: The fastest delivery in cricket history was bowled by Pakistani fast 

In [ ]:
chain.get_graph().print_ascii() # Visualize the chain (requires grandalf) 

     +-------------+       
     | PromptInput |       
     +-------------+       
            *              
            *              
            *              
    +----------------+     
    | PromptTemplate |     
    +----------------+     
            *              
            *              
            *              
      +----------+         
      | ChatGroq |         
      +----------+         
            *              
            *              
            *              
   +-----------------+     
   | StrOutputParser |     
   +-----------------+     
            *              
            *              
            *              
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  


##### **Sequential Chains**


In [5]:
prompt1 = PromptTemplate(
    template="Generate a detailed report about the {topic}",
    input_variables=['topic']
)

prompt2 = PromptTemplate(
    template="Generate a detailed summary of the report {text}",
    input_variables=['text']
)

model = ChatGroq(model="llama-3.3-70b-versatile")
parser = StrOutputParser()

chain1 = prompt1 | model | parser | prompt2 | model | parser
result = chain1.invoke({'topic': 'Indian Political Scenario'})
print(result)

**Detailed Summary: Indian Political Scenario: An Overview**

The Indian political landscape is a complex and dynamic system, shaped by the country's rich history, diverse culture, and the aspirations of its 1.3 billion people. As the world's largest democracy, India has a unique system of governance, with a parliamentary form of government and a federal structure. This report provides an in-depth analysis of the current Indian political scenario, including major political parties, key issues, emerging trends, and state-level politics.

**Major Political Parties:**

The report highlights the following major political parties in India:

1. **Bharatiya Janata Party (BJP)**: The ruling party at the Center, led by Prime Minister Narendra Modi, which has been in power since 2014.
2. **Indian National Congress (INC)**: The main opposition party, led by President Rahul Gandhi, which has a strong presence in several states.
3. **All India Trinamool Congress (AITC)**: A regional party based in 

##### **Parallel Chains**
Uses the concept of Runnable Parallel

In [6]:
from langchain_core.runnables import RunnableParallel

prompt1 = PromptTemplate(
    template="Generate short & simple notes from the following text: \n {text}",
    input_variables=['text']
)

prompt2 = PromptTemplate(
    template="Generate 5 short question answers from the following text: \n {text}",
    input_variables=['text']
)

prompt3 = PromptTemplate(
    template="Merge the provided notes and quiz into a single document \n notes -> {notes} and quiz -> {quiz}",
    input_variables=['notes', 'quiz']
)

model1 = ChatGroq(model="llama-3.3-70b-versatile")
model2 = ChatGroq(model="openai/gpt-oss-20b")

parser = StrOutputParser()

parallel_chain = RunnableParallel({
    "notes": prompt1 | model1 | parser,
    "quiz": prompt2 | model2 | parser
})

merge_chain = prompt3 | model1 | parser

chain = parallel_chain | merge_chain

text = """
Support vector machines (SVMs) are a set of supervised learning methods used for classification, regression and outliers detection.

The advantages of support vector machines are:

Effective in high dimensional spaces.

Still effective in cases where number of dimensions is greater than the number of samples.

Uses a subset of training points in the decision function (called support vectors), so it is also memory efficient.

Versatile: different Kernel functions can be specified for the decision function. Common kernels are provided, but it is also possible to specify custom kernels.

The disadvantages of support vector machines include:

If the number of features is much greater than the number of samples, avoid over-fitting in choosing Kernel functions and regularization term is crucial.

SVMs do not directly provide probability estimates, these are calculated using an expensive five-fold cross-validation (see Scores and probabilities, below).

The support vector machines in scikit-learn support both dense (numpy.ndarray and convertible to that by numpy.asarray) and sparse (any scipy.sparse) sample vectors as input. However, to use an SVM to make predictions for sparse data, it must have been fit on such data. For optimal performance, use C-ordered numpy.ndarray (dense) or scipy.sparse.csr_matrix (sparse) with dtype=float64.
"""

result = chain.invoke({'text': text})
print(result)

**Support Vector Machines (SVMs) Notes and Quiz**

### Advantages of SVMs

1. **Effective in high dimensional spaces**: SVMs remain effective even when the number of dimensions exceeds the number of samples.
2. **Memory efficient**: SVMs use only a subset of training points—support vectors—in the decision function, making them memory efficient.
3. **Versatile with different kernel functions**: SVMs can be used with various kernel functions, allowing for flexibility in modeling different types of data.
4. **Works well with limited samples**: SVMs can perform well even with a limited number of samples.

### Disadvantages of SVMs

1. **Over-fitting possible with many features**: SVMs can suffer from over-fitting when dealing with a large number of features.
2. **No direct probability estimates**: SVMs do not provide direct probability estimates, requiring additional methods for estimation.
3. **Requires specific data format for optimal performance**: SVMs require a specific data format fo

In [7]:
chain.get_graph().print_ascii()

          +---------------------------+            
          | Parallel<notes,quiz>Input |            
          +---------------------------+            
                ***             ***                
              **                   **              
            **                       **            
+----------------+              +----------------+ 
| PromptTemplate |              | PromptTemplate | 
+----------------+              +----------------+ 
          *                             *          
          *                             *          
          *                             *          
    +----------+                  +----------+     
    | ChatGroq |                  | ChatGroq |     
    +----------+                  +----------+     
          *                             *          
          *                             *          
          *                             *          
+-----------------+            +-----------------+ 
| StrOutputP

##### **Conditional Chain**  
Uses Runnable branch - format ->  
    RunnableBranch(  
        (condition1, chain1),  
        (condition2, chain2),
        default chain  
    )

In [15]:
from langchain_core.runnables import RunnableBranch, RunnableLambda
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import Literal

model = ChatGroq(model="llama-3.3-70b-versatile")
parser1 = StrOutputParser()

class Feedback(BaseModel):

    sentiment: Literal['positive', 'negative'] = Field(description="The sentiment of the feedback")

parser2 = PydanticOutputParser(pydantic_object=Feedback)

# We want the LLM model to return only positive or negative sentiment that's why we are using PydanticOutputParser
prompt1 = PromptTemplate(
    template="Classify the sentiment of the following feedback text into postive or negative \n {feedback} \n {format_instruction}",
    input_variables=['feedback'],
    partial_variables={'format_instruction': parser2.get_format_instructions()}
)

classifier_chain = prompt1 | model | parser2

prompt2 = PromptTemplate(
    template = "Write an appropriate response to this positive feedback: \n {feedback} \n {format_instruction}",
    input_variables=['feedback'],
)

# positive_chain = prompt2 | model | parser1

prompt3 = PromptTemplate(
    template = "Write an appropriate response to this negative feedback: \n {feedback} \n {format_instruction}",
    input_variables=['feedback'],
)

# negative_chain = prompt3 | model | parser1

conditional_chain = RunnableBranch(
    (lambda x: x.sentiment == 'positive', prompt2 | model | parser1),
    (lambda x: x.sentiment == 'negative', prompt3 | model | parser1),
    RunnableLambda(lambda x: "Could not understand the feedback")
)

chain = classifier_chain | conditional_chain

result1 = chain.invoke({'feedback': 'The product is great and I love it!'})
print(result1)

result2 = chain.invoke({'feedback': 'The product is not good and I hate it!'})
print(result2)

TypeError: Expected mapping type as input to PromptTemplate. Received <class '__main__.Feedback'>.
For troubleshooting, visit: https://docs.langchain.com/oss/python/langchain/errors/INVALID_PROMPT_INPUT 

In [ ]:
from langchain_groq import ChatGroq
from langchain_anthropic import ChatAnthropic
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnableBranch, RunnableLambda
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import Literal

load_dotenv()

model = ChatGroq(model="llama-3.3-70b-versatile")

parser = StrOutputParser()

class Feedback(BaseModel):

    sentiment: Literal['positive','negative'] = Field(description='Give the sentiment of the feedback')

parser2 = PydanticOutputParser(pydantic_object=Feedback)

prompt1 = PromptTemplate(
    template='Classify the sentiment of the following feedback text into postive or negative \n {feedback} \n {format_instruction}',
    input_variables=['feedback'],
    partial_variables={'format_instruction':parser2.get_format_instructions()}
)

classifier_chain = prompt1 | model | parser2

prompt2 = PromptTemplate(
    template='Write an appropriate response to this positive feedback \n {feedback}',
    input_variables=['feedback']
)

prompt3 = PromptTemplate(
    template='Write an appropriate response to this negative feedback \n {feedback}',
    input_variables=['feedback']
)

branch_chain = RunnableBranch(
    (lambda x:x.sentiment == 'positive', prompt2 | model | parser),
    (lambda x:x.sentiment == 'negative', prompt3 | model | parser),
    RunnableLambda(lambda x: "could not find sentiment")
)

chain = classifier_chain | branch_chain

print(chain.invoke({'feedback': 'This is a beautiful phone'}))


Thank you so much for your kind words. We're thrilled to hear that you're happy with our service. Your satisfaction is our top priority, and we're glad we could meet your expectations. If you have any other questions or need further assistance, don't hesitate to reach out. We appreciate your feedback and look forward to serving you again in the future.
